In [2]:
import os
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
import torchmetrics
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from PIL import Image

In [3]:
# --------------------
# Dataset Paths
# --------------------
TRAIN_DIR = "/kaggle/input/fruit-and-vegetable-image-recognition/train"      # adapt to your dataset
VAL_DIR   = "/kaggle/input/fruit-and-vegetable-image-recognition/validation" # adapt to your dataset
TEST_DIR  = "/kaggle/input/fruit-and-vegetable-image-recognition/test"       # adapt to your dataset
NUM_CLASSES = 36  # adapt if needed

In [ ]:
#!nvidia-smi

In [4]:
# --------------------
# Custom Loader
# --------------------
def load_rgb_image(path):
    """Ensures images are loaded in RGB format (handles 'P' or 'RGBA')."""
    img = Image.open(path)
    if img.mode == 'P':
        img = img.convert("RGBA").convert("RGB")
    elif img.mode == 'RGBA':
        img = img.convert("RGB")
    return img

In [5]:
# --------------------
# Transforms
# --------------------
def build_transforms(img_size=150):
    return transforms.Compose([
        transforms.Lambda(lambda img: img.convert("RGB") if img.mode != "RGB" else img),
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])

In [6]:
# --------------------
# Simple CNN Model
# --------------------
class SimpleCNN(nn.Module):
    """A simple CNN for demonstration."""
    def __init__(
        self, 
        num_conv_blocks=2, 
        filters=32, 
        kernel_size=3, 
        dropout_rate=0.0, 
        num_classes=36,
        image_size=150
    ):
        super().__init__()
        self.layers = nn.ModuleList()
        in_channels = 3
        for i in range(num_conv_blocks):
            out_channels = filters * (2 ** i)
            self.layers.append(nn.Conv2d(in_channels, out_channels, kernel_size, padding="same"))
            self.layers.append(nn.ReLU())
            self.layers.append(nn.MaxPool2d(2))
            in_channels = out_channels

        # compute final flatten size
        factor = 2 ** num_conv_blocks
        flat_dim = out_channels * (image_size // factor) * (image_size // factor)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(flat_dim, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        return self.fc2(x)


In [7]:

# --------------------
# Lightning Module
# --------------------
class LitModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        # read hyperparams from wandb.config
        self.num_conv_blocks = wandb.config.num_conv_blocks
        self.filters         = wandb.config.filters
        self.kernel_size     = wandb.config.kernel_size
        self.dropout_rate    = wandb.config.dropout_rate
        self.learning_rate   = wandb.config.learning_rate
        self.image_size      = wandb.config.image_size
        self.num_classes     = NUM_CLASSES

        # define model
        self.model = SimpleCNN(
            num_conv_blocks=self.num_conv_blocks,
            filters=self.filters,
            kernel_size=self.kernel_size,
            dropout_rate=self.dropout_rate,
            num_classes=self.num_classes,
            image_size=self.image_size
        )

        self.criterion = nn.CrossEntropyLoss()

        # metrics
        self.train_acc = torchmetrics.Accuracy(task='multiclass', num_classes=self.num_classes)
        self.val_acc   = torchmetrics.Accuracy(task='multiclass', num_classes=self.num_classes)
        self.test_acc  = torchmetrics.Accuracy(task='multiclass', num_classes=self.num_classes)

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.learning_rate)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss   = self.criterion(logits, y)
        preds  = torch.argmax(logits, dim=1)
        self.train_acc.update(preds, y)

        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log("train_acc", self.train_acc, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss   = self.criterion(logits, y)
        preds  = torch.argmax(logits, dim=1)
        self.val_acc.update(preds, y)

        self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log("val_acc", self.val_acc, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss   = self.criterion(logits, y)
        preds  = torch.argmax(logits, dim=1)
        self.test_acc.update(preds, y)

        self.log("test_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log("test_acc", self.test_acc, on_step=True, on_epoch=True, prog_bar=True)
        return loss

In [8]:
# --------------------
# Train Function
# --------------------
def train():
    wandb.init()  # each run of the sweep is a new W&B run

    # read more config
    batch_size   = wandb.config.batch_size
    image_size   = wandb.config.image_size
    max_epochs   = wandb.config.max_epochs
    num_workers  = wandb.config.num_workers

    # data transforms
    transform = build_transforms(img_size=image_size)

    # datasets
    train_ds = datasets.ImageFolder(TRAIN_DIR, loader=load_rgb_image, transform=transform)
    val_ds   = datasets.ImageFolder(VAL_DIR,   loader=load_rgb_image, transform=transform)
    test_ds  = datasets.ImageFolder(TEST_DIR,  loader=load_rgb_image, transform=transform)

    # loaders
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, 
                              num_workers=num_workers, pin_memory=True)
    val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, 
                              num_workers=num_workers, pin_memory=True)
    test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False,
                              num_workers=num_workers, pin_memory=True)

    # define model & trainer
    model = LitModel()
    wandb_logger = WandbLogger(entity="25DLAAU", project="FruitVeg-RandomSearch", log_model=False)
    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator="auto",
        devices=1,
        logger=wandb_logger
    )

    # fit & test
    trainer.fit(model, train_loader, val_loader)
    trainer.test(model, test_loader)

    wandb.finish()


In [10]:
# --------------------
# Sweep Config
# --------------------
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
            'values': [1e-4, 1e-3, 1e-2]
        },
        'dropout_rate': {
            'min': 0.0,
            'max': 0.5
        },
        'num_conv_blocks': {
            'values': [2, 3]
        },
        'filters': {
            'values': [16, 32]
        },
        'kernel_size': {
            'values': [3, 5]
        },
        'batch_size': {
            'value': 32
        },
        'image_size': {
            'value': 150
        },
        'max_epochs': {
            'value': 5
        },
        'num_workers': {
            'value': 2
        }
    }
}

In [12]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Wandb")
os.environ["WANDB_API_KEY"] = secret_value_0

In [19]:
# --------------------
# Run Everything
# --------------------

# 1) Log in to wandb
wandb.login(key=secret_value_0)  # make sure you are logged in; may prompt or use your env key

# 2) Create the sweep
sweep_id = wandb.sweep(
    sweep=sweep_config, 
    project="FruitVeg-RandomSearch", 
    entity="25DLAAU"
)
print("Create sweep with ID:", sweep_id)

wandb: Currently logged in as: ktorta24 (ktorta24-aalborg-universitet). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: iiiiem3n
Sweep URL: https://wandb.ai/25DLAAU/FruitVeg-RandomSearch/sweeps/iiiiem3n
Create sweep with ID: iiiiem3n


In [20]:
wandb.agent(sweep_id, function=train, count=3)

wandb: Agent Starting Run: ptwsttpp with config:
wandb: 	batch_size: 32
wandb: 	dropout_rate: 0.0017035784500765705
wandb: 	filters: 32
wandb: 	image_size: 150
wandb: 	kernel_size: 3
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 5
wandb: 	num_conv_blocks: 3
wandb: 	num_workers: 2
wandb: Currently logged in as: ktorta24 (25DLAAU). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │     0.902506947517395     │
│      test_loss_epoch      │    0.4173075556755066     │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▂▂▂▂▄▄▄▄▅▅▅▅▇▇▇▇█
test_acc_epoch,▁
test_loss_epoch,▁
test_loss_step,█▂▄▂▂▂▃▁▇▁▆▄
train_acc_epoch,▁▃▅▆█
train_loss_epoch,█▅▄▃▁
train_loss_step,█▇▆▆█▄▃▂▁
trainer/global_step,▂▁▁▁▁▁▁▃▃▁▁▁▄▄▅▁▁▁▁▂▂▆▆▇▂▂▂▂██▂▂▂▂▂▂▁▁▁▁
val_acc_epoch,▁▄▅▇█
val_loss_epoch,█▅▄▂▁
val_loss_step,▇▇▇█▇▅█▆▅▅▆▄▅▄▅▄▅▃▆▄▄▅▄▄▃▄▃▂▂▂▄▃▃▃▁▂▂▁▃▂
